# Keras入门
Keras是一个非常流行、简单的深度学习框架，它的设计参考了torch，用Python语言编写，是一个高度模块化的神经网络库，支持GPU和CPU。能够在TensorFlow，CNTK或Theano之上运行。 Keras的特点是能够快速实现模型的搭建， 简单方便地让你实现从想法到实验验证的转化，这都是高效地进行科学研究的关键。  

本文使用MNIST实例介绍Keras入门

In [2]:
from keras.models import *
from keras.layers import Conv2D, BatchNormalization, Activation, Flatten, Dense

## 模型定义

In [3]:
def simpleConv3(input_shape=(48, 48, 3), classes=2):
    img_input = Input(shape=input_shape)
    bn_axis = 3
    x = Conv2D(12, (3,3), strides=(2,2), padding='same', name='conv1')(img_input)
    # 输出是12通道，卷积核大小3*3，步长为2，padding='same'表示边缘补零
    x = BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
    # axis表示需要归一化的坐标轴，bn_axis=3，由于采用TensorFlow作为后端，因此这句代码表示在通道数坐标轴进行归一化。
    x = Activation('relu')(x)
    
    x = Conv2D(24, (3, 3), strides=(2, 2), padding='same', name='conv2')(x)
    x = BatchNormalization(axis=bn_axis, name='bn_conv2')(x)
    x = Activation('relu')(x)

    x = Conv2D(48, (3, 3), strides=(2, 2), padding='same', name='conv3')(x)
    x = BatchNormalization(axis=bn_axis, name='bn_conv3')(x)
    x = Activation('relu')(x)
    
    x = Flatten()(x)
    # 将卷积特征图进行拉伸，以便和全连接层Dense()进行连接。
    x = Dense(128, activation='relu')(x)
    # 实现全连接层的功能，1200是输出维度，‘relu'表示激活函数
    x = Dense(classes, activation='softmax')(x)
    # 采用‘softmax’激活函数实现分类功能
    
    model = Model(img_input, x)
    # 返回Model，包含网络的输入和输出
    return model

## 模型编译
网络搭建完成，在网络训练前需要进行编译，包括学习方法、损失函数、评估标准等，这些参数分别可以从optimizer、loss、metric模块中导入。

In [4]:
from keras.optimizers import SGD
from keras.losses import binary_crossentropy
from keras.metrics import binary_accuracy
from keras.callbacks import TensorBoard

tensorboard = TensorBoard(log_dir=('./logs'))
callbacks = []
callbacks.append(tensorboard)
loss = binary_crossentropy
metrics = [binary_accuracy]
optimizer = SGD(lr=0.001, decay=1e-6, momentum=0.9)

## 模型训练

In [ ]:
def train_model(model, loss, metrics, optimizer, epochs=25):

    model.compile(loss=loss,
                  optimizer=optimizer,
                  metrics=metrics)
    model.summary()

    history = model.fit_generator(
        train_generator,
        steps_per_epoch=num_train_samples // batch_size,
        epochs=epochs,
        callbacks=callbacks,
        validation_data=val_generator,
        validation_steps=num_val_samples // batch_size)
    return model

## 模型运行

In [ ]:

if __name__ == '__main__':
    # dimensions of our images.
    img_width, img_height = 48, 48
    num_epochs = 25
    batch_size = 16

    train_data_dir = 'data/train/'
    validation_data_dir = 'data/val'

    # this is the augmentation configuration we will use for training
    train_datagen = ImageDataGenerator(
        rescale=1. / 255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
    # this is the augmentation configuration use for testing only rescaling
    val_datagen = ImageDataGenerator(rescale=1. / 255)

    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size)

    val_generator = val_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size)

    num_train_samples = train_generator.samples
    num_val_samples = val_generator.samples

    tensorboard = TensorBoard(log_dir=('./logs'))
    callbacks = []
    callbacks.append(tensorboard)
    model = simpleconv3()
    loss = binary_crossentropy
    metrics = [binary_accuracy]
    optimizer = SGD(lr=0.001, decay=1e-6, momentum=0.9)

    model = train_model(model, loss, metrics,  optimizer, num_epochs)
    if not os.path.exists('models'):
        os.mkdir('models')
    model.save_weights('models/model.h5')

## 模型测试

In [ ]:
import sys
import cv2
import numpy as np
import tensorflow as tf
import keras.backend as K

from net import simpleconv3
from PIL import Image
from keras.preprocessing.image import img_to_array
from keras.applications.imagenet_utils import preprocess_input

image_size = (48, 48)
batch_shape = (1, ) + image_size + (3, )
model_path = sys.argv[1]
# model_path = './models/model.h5'

config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
session = tf.Session(config=config)
K.set_session(session)

model = simpleconv3()
model.load_weights(model_path, by_name=True)
model.summary()

image_path = sys.argv[2]
# image_path = '../../../../datas/head/train/0/1left.jpg'
img = Image.open(image_path)
img = img_to_array(img)
img = cv2.resize(img, image_size)
img = np.expand_dims(img, axis=0)
img = preprocess_input(img)

result = model.predict(img, batch_size=1)
print(result)